# Ristiintaulukoinnit Exceliin

In [1]:
import pandas as pd

# Avaan datan dataframeen
df = pd.read_excel('http://taanila.fi/data1.xlsx')
df.head()

,nro,sukup,ikä,perhe,koulutus,palveluv,palkka,johto,työtov,työymp,palkkat,työteht,työterv,lomaosa,kuntosa,hieroja
0,1,1,38,1,1.0,22.0,3587,3,3.0,3,3,3,NaN,NaN,NaN,NaN
1,2,1,29,2,2.0,10.0,2963,1,5.0,2,1,3,NaN,NaN,NaN,NaN
2,3,1,30,1,1.0,7.0,1989,3,4.0,1,1,3,1.0,NaN,NaN,NaN
3,4,1,36,2,1.0,14.0,2144,3,3.0,3,3,3,1.0,NaN,NaN,NaN
4,5,1,24,1,2.0,4.0,2183,2,3.0,2,1,2,1.0,NaN,NaN,NaN


In [2]:
# Muodostan uuden dataframen ristiintaulukoitavista muuttujista
columns = ['sukup', 'perhe', 'koulutus', 'johto', 'työtov',
        'työymp', 'palkkat', 'työteht']
df1 = pd.DataFrame(df, columns = columns)

# Korvaan muuttujien numeroarvoja oikeilla nimillä
df1['sukup'] = df1['sukup'].replace([1, 2], ['Mies', 'Nainen'])
df1['perhe'] = df1['perhe'].replace([1, 2], ['Perheetön', 'Perheellinen'])

In [3]:
# Alustan ExcelWriterin kirjoittamaan uuteen tiedostoon ristit.xlsx
writer = pd.ExcelWriter('ristiintaulukoinnit.xlsx', engine = 'xlsxwriter')

# Määritän prosenttimuotoilun
format = writer.book.add_format({'num_format': '0.0 %'})

# Pidän kirjaa rivinumerosta rivi-muuttujan avulla
rivi = 0

# for-silmukoilla käyn läpi kaikki muuttujaparit
for var1 in df1:
    for var2 in df1:
        if var1 != var2:
            
            # Lasken ristiintaulukoinnin df2-nimiseen dataframeen
            df2 = pd.crosstab(df1[var1], df1[var2])
                                                
            # Lasken ristiintaulukoinnin prosentteina df3-nimiseen dataframeen
            df3 = pd.crosstab(df1[var1], df1[var2], normalize='columns')
            
            # df3:n vasempaan nurkkaan muuttujien nimet
            df3.index.name=var1+'/'+var2 
            
            # Lisään df3:een summarivin
            df3.loc['n'] = df2.sum()
                                    
            # Kirjoitan df3:n  Ristiintaulukoinnit-taulukkovälilehdelle
            df3.to_excel(writer, sheet_name = 'Ristiintaulukoinnit', startrow = rivi)
            
            # Prosenttimuotoilut
            for i in range(rivi+1, rivi+df3.shape[0]):
                writer.sheets['Ristiintaulukoinnit'].set_row(i, cell_format=format)
            
            # Kasvatan rivinumeroa; shape[0] antaa rivimäärän (shape[1] antaisi sarakkeiden määrän)
            rivi = rivi + df3.shape[0] + 2

# Tallennan Excel-tiedoston
writer.save()

Tämän jälkeen voin avata ristiintaulukoinnit.xlsx -tiedoston Exceliin ja katsoa mitä sain aikaan.